In [1]:
import mlflow
mlflow.set_tracking_uri("http://13.221.75.66:5000/")

c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [2]:
import mlflow

# Set experiment before logging anything
mlflow.set_experiment("ML Algos with HPT")

with mlflow.start_run():
    mlflow.log_param("example_param", 42)


KeyboardInterrupt: 

In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd

In [4]:
df= pd.read_csv(r'C:\Users\Deepu\OneDrive\Desktop\Capstone1\reddit_preprocessing.csv').dropna()
df.shape

(36750, 7)

In [21]:
# Instantiate LightGBM classifier correctly
lgb = LGBMClassifier()

In [22]:
X_cleaned = df['clean_comment']
y_cleaned = df['category'] 

In [23]:
# Step 5: Train-test split
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42
)


In [24]:
ngram_range = (1, 3)  # Trigram
max_features = 10000
TF_IDF_cleaned = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

In [25]:
X_train_tfidf_cleaned = TF_IDF_cleaned.fit_transform(X_train_cleaned)
X_test_tfidf_cleaned = TF_IDF_cleaned.transform(X_test_cleaned)

In [27]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [28]:
# Function to optimize LightGBM hyperparameters
def objective(trial):
    # Define hyperparameters to be tuned
    param = {
        "objective": "multiclass",
        "num_class": 3,  # Assuming 3 categories (-1, 0, 1)
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced"
    }

    # Define the LightGBM model with the trial parameters
    model = lgb.LGBMClassifier(**param)

    # Perform cross-validation
    scores = cross_val_score(model, X_train_tfidf_cleaned, y_train_cleaned, cv=3, scoring='accuracy')

    # Return the average score across folds
    return scores.mean()


In [29]:
# Create an Optuna study to optimize the hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Extract the best hyperparameters
best_params = study.best_params
print(best_params)

[I 2025-08-15 15:22:05,763] A new study created in memory with name: no-name-7f3abf6b-b25f-4f78-93df-8fc00000c9bd
c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.p

{'learning_rate': 0.09784602901436658, 'n_estimators': 482, 'max_depth': 17}


In [38]:
best_params = study.best_params
best_params

{'learning_rate': 0.09784602901436658, 'n_estimators': 482, 'max_depth': 17}

In [32]:
# Create the best model using tuned hyperparameters
best_model = LGBMClassifier(
    objective="multiclass",
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=0.1,  # L2 regularization
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    n_estimators=best_params["n_estimators"]
)

In [33]:
# Fit the model on the resampled training data
best_model.fit(X_train_tfidf_cleaned, y_train_cleaned)

# Predict on the train set
y_train_pred = best_model.predict(X_train_tfidf_cleaned)

# Calculate accuracy on the train set
accuracy_train = accuracy_score(y_train_cleaned, y_train_pred)
print("Train Accuracy:", accuracy_train)

c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Train Accuracy: 0.9902635849304855


In [34]:
# Generate classification report
report_train = classification_report(y_train_cleaned, y_train_pred)
print(report_train)

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     10169
         1.0       1.00      0.98      0.99     12632

    accuracy                           0.99     22801
   macro avg       0.99      0.99      0.99     22801
weighted avg       0.99      0.99      0.99     22801



In [36]:
import numpy as np 
import re 

# Function to clean and preprocess a YouTube comment (same as used during training)
def preprocess_comment(comment):
    # Lowercasing
    comment = comment.lower()

    # Remove special characters, URLs, punctuation, and extra spaces
    comment = re.sub(r"http\S+|www\S+|https\S+", ' ', comment, flags=re.MULTILINE)  # Remove URLs
    comment = re.sub(r"\W", " ", comment)  # Remove special characters
    comment = re.sub(r"\s+", " ", comment).strip()  # Remove extra spaces and newlines

    return comment


In [37]:
# Prediction function
def predict_sentiment(comment, tfidf_vectorizer, lgbm_model):
    # Step 1: Preprocess the YouTube comment
    cleaned_comment = preprocess_comment(comment)

    # Step 2: Transform the comment using the trained TF-IDF vectorizer
    comment_tfidf = tfidf_vectorizer.transform([cleaned_comment])

    # Step 3: Use the trained LightGBM model to predict the sentiment
    prediction = lgbm_model.predict(comment_tfidf)
    prediction_proba = lgbm_model.predict_proba(comment_tfidf)

    # Step 4: Get the predicted sentiment (label) and probability
    sentiment_class = np.argmax(prediction_proba)
    sentiment_proba = np.max(prediction_proba)

    # Step 5: Return the sentiment label and confidence
    return {
        'sentiment_class': int(prediction[0]),  # e.g., -1, 0, or 1 depending on your labels
        'confidence': sentiment_proba
    }

# Example usage:
comment = "I absolutely hate this video!"
result = predict_sentiment(comment, TF_IDF_cleaned, best_model)
print(result)


{'sentiment_class': 1, 'confidence': 0.8871236069577338}


c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
